--- Day 13: Distress Signal ---

You climb the hill and again try contacting the Elves. However, you instead receive a signal you weren't expecting: a **distress signal**.

Your handheld device must still not be working properly; the packets from the distress signal got decoded **out of order**. You'll need to re-order the list of received packets (your puzzle input) to decode the message.

Your list consists of pairs of packets; pairs are separated by a blank line. You need to identify **how many pairs of packets are in the right order**.

For example:
```
[1,1,3,1,1]
[1,1,5,1,1]

[[1],[2,3,4]]
[[1],4]

[9]
[[8,7,6]]

[[4,4],4,4]
[[4,4],4,4,4]

[7,7,7,7]
[7,7,7]

[]
[3]

[[[]]]
[[]]

[1,[2,[3,[4,[5,6,7]]]],8,9]
[1,[2,[3,[4,[5,6,0]]]],8,9]
```

Packet data consists of lists and integers. Each list starts with [, ends with ], and contains zero or more comma-separated values (either integers or other lists). Each packet is always a list and appears on its own line.

When comparing two values, the first value is called **left** and the second value is called **right**. Then:

- If **both values are integers**, the **lower integer** should come first. If the left integer is lower than the right integer, the inputs are in the right order. If the left integer is higher than the right integer, the inputs are not in the right order. Otherwise, the inputs are the same integer; continue checking the next part of the input.
- If **both values are lists**, compare the first value of each list, then the second value, and so on. If the left list runs out of items first, the inputs are in the right order. If the right list runs out of items first, the inputs are not in the right order. If the lists are the same length and no comparison makes a decision about the order, continue checking the next part of the input.
- If **exactly one value is an integer**, convert the integer to a list which contains that integer as its only value, then retry the comparison. For example, if comparing `[0,0,0]` and `2`, convert the right value to `[2]` (a list containing `2`); the result is then found by instead comparing `[0,0,0]` and `[2]`.
Using these rules, you can determine which of the pairs in the example are in the right order:
```
== Pair 1 ==
- Compare [1,1,3,1,1] vs [1,1,5,1,1]
  - Compare 1 vs 1
  - Compare 1 vs 1
  - Compare 3 vs 5
    - Left side is smaller, so inputs are in the right order

== Pair 2 ==
- Compare [[1],[2,3,4]] vs [[1],4]
  - Compare [1] vs [1]
    - Compare 1 vs 1
  - Compare [2,3,4] vs 4
    - Mixed types; convert right to [4] and retry comparison
    - Compare [2,3,4] vs [4]
      - Compare 2 vs 4
        - Left side is smaller, so inputs are in the right order

== Pair 3 ==
- Compare [9] vs [[8,7,6]]
  - Compare 9 vs [8,7,6]
    - Mixed types; convert left to [9] and retry comparison
    - Compare [9] vs [8,7,6]
      - Compare 9 vs 8
        - Right side is smaller, so inputs are not in the right order

== Pair 4 ==
- Compare [[4,4],4,4] vs [[4,4],4,4,4]
  - Compare [4,4] vs [4,4]
    - Compare 4 vs 4
    - Compare 4 vs 4
  - Compare 4 vs 4
  - Compare 4 vs 4
  - Left side ran out of items, so inputs are in the right order

== Pair 5 ==
- Compare [7,7,7,7] vs [7,7,7]
  - Compare 7 vs 7
  - Compare 7 vs 7
  - Compare 7 vs 7
  - Right side ran out of items, so inputs are not in the right order

== Pair 6 ==
- Compare [] vs [3]
  - Left side ran out of items, so inputs are in the right order

== Pair 7 ==
- Compare [[[]]] vs [[]]
  - Compare [[]] vs []
    - Right side ran out of items, so inputs are not in the right order

== Pair 8 ==
- Compare [1,[2,[3,[4,[5,6,7]]]],8,9] vs [1,[2,[3,[4,[5,6,0]]]],8,9]
  - Compare 1 vs 1
  - Compare [2,[3,[4,[5,6,7]]]] vs [2,[3,[4,[5,6,0]]]]
    - Compare 2 vs 2
    - Compare [3,[4,[5,6,7]]] vs [3,[4,[5,6,0]]]
      - Compare 3 vs 3
      - Compare [4,[5,6,7]] vs [4,[5,6,0]]
        - Compare 4 vs 4
        - Compare [5,6,7] vs [5,6,0]
          - Compare 5 vs 5
          - Compare 6 vs 6
          - Compare 7 vs 0
            - Right side is smaller, so inputs are not in the right order
```
What are the indices of the pairs that are already **in the right order**? (The first pair has index `1`, the second pair has index `2`, and so on.) In the above example, the pairs in the right order are `1`, `2`, `4`, and `6`; the sum of these indices is **`13`**.

Determine which pairs of packets are already in the right order. **What is the sum of the indices of those pairs?**

In [1]:
# Data Load
example_lines = [
    "[1,1,3,1,1]",
    "[1,1,5,1,1]",
    "",
    "[[1],[2,3,4]]",
    "[[1],4]",
    "",
    "[9]",
    "[[8,7,6]]",
    "",
    "[[4,4],4,4]",
    "[[4,4],4,4,4]",
    "",
    "[7,7,7,7]",
    "[7,7,7]",
    "",
    "[]",
    "[3]",
    "",
    "[[[]]]",
    "[[]]",
    "",
    "[1,[2,[3,[4,[5,6,7]]]],8,9]",
    "[1,[2,[3,[4,[5,6,0]]]],8,9]",
]

with open("day_13_input.txt") as _file:
    data_lines = _file.readlines()

In [2]:
# Common Code
from __future__ import annotations
import typing


class Packet:
    _LIST_START = "["
    _LIST_STOP = "]"
    _LIST_SEPARATOR = ","

    def __init__(self, values: list[typing.Union[int, Packet]]):
        self._values = values

    def __repr__(self):
        # return self._values
        return "[" + ",".join([str(x) if isinstance(x, int) else x.__repr__() for x in self._values]) + "]"

    @classmethod
    def _parse_list(cls, line_segment: str) -> (Packet, int):
        values = []
        next_value = None

        def _add_value(value):
            if isinstance(value, str):
                value = int(value)

            if value is not None:
                values.append(value)

        if line_segment[0] != cls._LIST_START:
            raise RuntimeError(f"Invalid starting input. Expected {cls._LIST_START}")

        i = 1
        while i < len(line_segment) and line_segment[i] != cls._LIST_STOP:
            symbol = line_segment[i]

            if symbol == cls._LIST_START:
                next_value, list_length = cls._parse_list(line_segment[i:])
                i += list_length
            elif symbol == cls._LIST_SEPARATOR:
                _add_value(next_value)
                next_value = None
            else:
                if next_value is None:
                    next_value = ""
                next_value += symbol

            i += 1

        _add_value(next_value)

        return Packet(values), i

    @classmethod
    def parse(cls, line: str) -> Packet:
        packet, _ = cls._parse_list(line.strip())
        return packet

    def _cmp(self, other: Packet):
        other_length = len(other._values)

        for i, left in enumerate(self._values):
            if i >= other_length:
                return 1

            right = other._values[i]

            left_is_int = isinstance(left, int)
            right_is_int = isinstance(right, int)

            if left_is_int and right_is_int:
                if left < right:
                    return -1
                elif left > right:
                    return 1
            else:
                if left_is_int:
                    left = Packet([left])
                elif right_is_int:
                    right = Packet([right])

                comparison = left._cmp(right)
                if comparison != 0:
                    return comparison

        return 0 if len(self._values) == other_length else -1

    def __lt__(self, other: Packet):
        return self._cmp(other) < 0


class PacketPair:
    def __init__(self, first, second):
        self._first = first
        self._second = second

    @classmethod
    def parse_pairs(cls, lines) -> list[PacketPair]:
        packets = []
        for line in lines:
            line = line.strip()
            if line:
                packets.append(Packet.parse(line))

        pairs = []
        for i in range(0, len(packets), 2):
            pairs.append(cls(packets[i], packets[i+1]))

        return pairs

    @property
    def in_order(self):
        return self._first < self._second

In [3]:
# Solution 1 Code
def solution_1(lines):
    pairs = PacketPair.parse_pairs(lines)

    ordered_pair_indices = [i for i, x in enumerate(pairs) if x.in_order]

    return sum(x + 1 for x in ordered_pair_indices)

def test_solution_1():
    actual = solution_1(example_lines)
    expected = 13
    
    print(actual, expected)
    assert actual == expected
test_solution_1()

13 13


In [4]:
# Solution 1
solution_1(data_lines)

4643

_Problem 2_

In [5]:
# Solution 2 Code
def solution_2(lines):
    return ""

def test_solution_2():
    actual = solution_2(example_lines)
    expected = ""
    
    print(actual, expected)
    assert actual == expected
test_solution_2()

In [6]:
# Solution 2
solution_2(data_lines)

''